# Домашнее задание по теме: "Основы веб-скрапинга"

## Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса.

Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [149]:
# импорт библиотек
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time
from IPython.display import display, HTML # для отображения полных строк и кликабельных ссылок
from random import random
import requests
import re

In [150]:
def get_posts_df(url, params):
    '''Функция получающая данные из поиска на сайте habr.ru'''
    df = pd.DataFrame()

    res = requests.get(url + f'/ru/search', params=params)
    soup = BeautifulSoup(res.text)

    # полученние объектов с классом tm-articles-list__item
    articles = soup.find_all('article', class_='tm-articles-list__item') 

    # проход в цикле по объектам articles, получение и запись нужных данных в dataframe
    for article in articles:
        date_full = article.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime')
        date = datetime.strptime(str(date_full), '%Y-%m-%dT%H:%M:%S.000Z') # изменение формата даты
        title = article.find('a', class_="tm-article-snippet__title-link").text
        link = url + article.find('a', class_="tm-article-snippet__title-link").get('href')

        # итоговый словарь для формирования строки
        row = {'date' : date, 'title' : title, 'link' : link}

        # построчное добавление данных в dataframe
        df = pd.concat([df, pd.DataFrame([row])])

    return df

In [152]:
# ссылка и параметры
url = 'https://habr.com'
params = {
   'q': 'python'
}

habr_posts = get_posts_df(url, params)

# проверка
display(HTML(habr_posts.head(10).to_html(render_links=True, escape=False)))

,date,title,link
0,2021-12-13 06:00:03,"Жаждущим автоматизации: открытый урок «ChatOps c Errbot на Python», 21 декабря",https://habr.com/ru/company/southbridge/news/t/595093/
0,2022-01-20 15:37:16,Курс «Python для инженеров». Старт 3 потока 31 января,https://habr.com/ru/company/southbridge/news/t/646825/
0,2020-04-21 15:35:14,"Вышел Python 2.7.18, последний релиз ветки Python 2.x",https://habr.com/ru/news/t/498364/
0,2021-07-06 10:29:57,Python Community Meetup 8/07: видео и материалы встречи,https://habr.com/ru/company/raiffeisenbank/news/t/566370/
0,2022-01-13 15:35:30,"Открытый урок «Пишем Custom Prometheus Exporter на Python», 19 января",https://habr.com/ru/company/southbridge/news/t/645485/
0,2020-12-04 18:03:25,Python как компилируемый статически типизированный язык программирования,https://habr.com/ru/news/t/531402/
0,2022-03-08 09:13:52,Вышел мартовский релиз расширения Python для Visual Studio Code,https://habr.com/ru/news/t/654707/
0,2020-03-03 10:22:32,В начале этого года Python сместил Java и стал вторым по популярности языком программирования среди разработчиков,https://habr.com/ru/company/itsumma/news/t/490834/
0,2021-11-16 13:09:46,EPAM разработала бесплатный курс по программированию для детей и подростков «Chatbot on Python. Part 1»,https://habr.com/ru/company/epam_systems/news/t/589555/
0,2022-08-19 15:07:02,Осталась неделя до старта 4 потока Python для инженеров,https://habr.com/ru/company/southbridge/news/t/683604/


## Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [153]:
def get_posts_df(url, params, pcount):
    '''Функция получающая данные из поиска на сайте habr.ru'''
    df = pd.DataFrame()

    for page in range(0, pcount):
        res = requests.get(url + f'/ru/search/page{page + 1}', params=params)
        soup = BeautifulSoup(res.text)

        # полученние объектов с классом tm-articles-list__item
        articles = soup.find_all('article', class_='tm-articles-list__item')

        # проход в цикле по объектам articles, получение и запись нужных данных в dataframe
        for article in articles:
            date_full = article.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime')
            date = datetime.strptime(str(date_full), '%Y-%m-%dT%H:%M:%S.000Z') # изменение формата даты
            title = article.find('a', class_="tm-article-snippet__title-link").text
            link = url + article.find('a', class_="tm-article-snippet__title-link").get('href')

            # получение текста материала
            text = BeautifulSoup(requests.get(link).text).find('div', class_='article-formatted-body').text

            # получение лайков и дизлайков
            votes_text = article.find('svg', class_='tm-votes-meter__icon').text
            votes = re.findall(r'\d+', votes_text)
            # если список пустой подстановка нулей
            likes, dislikes = (votes[1], votes[2]) if len(votes) != 0 else (0, 0)

            # итоговый словарь для формирования строки
            row = {'date' : date, 'title' : title, 'link' : link, 'text': text, 'likes': likes, 'dislikes': dislikes}

            # построчное добавление данных в dataframe
            df = pd.concat([df, pd.DataFrame([row])])

        # пауза, что бы избежать блокировку
        time.sleep(0) if pcount == 1 else time.sleep(random())

    return df

In [154]:
# ссылка и параметры
url = 'https://habr.com'
params = {
   'q': 'python'
}

# количество страниц
pcount = 5

habr_posts = get_posts_df(url, params, pcount)

# проверка
display(HTML(habr_posts.head(10).to_html(render_links=True, escape=False, formatters={'text': lambda x: str(x)[:256]})))

,date,title,link,text,likes,dislikes
0,2021-12-13 06:00:03,"Жаждущим автоматизации: открытый урок «ChatOps c Errbot на Python», 21 декабря",https://habr.com/ru/company/southbridge/news/t/595093/,"21 декабря Слёрм проведёт открытый урок «ChatOps c Errbot на Python». Вещает Антон Рязанцев – спикер курса «Python для инженеров».Начнём в 18:00 по Москве, действо продлится примерно один час, запись будет.Открытый урок «ChatOps c Errbot на Python»Поговори",10,1
0,2022-01-20 15:37:16,Курс «Python для инженеров». Старт 3 потока 31 января,https://habr.com/ru/company/southbridge/news/t/646825/,"Курс нацелен дать максимальную пользу, поэтому в нём нет абстрактных задач и любых проявлений Python для веба, аналитики и бекенда.Будет разбор библиотек Python для DevOps, правила эффективного и поддерживаемого кода, решение конкретных задач DevOps, напри",12,2
0,2020-04-21 15:35:14,"Вышел Python 2.7.18, последний релиз ветки Python 2.x",https://habr.com/ru/news/t/498364/,"20 апреля 2020 года, спустя почти десять лет после выхода Python 2.7, выпущен «на волю» Python версии 2.7.18. Это последний релиз ветки Python 2.x, поддерживаемый параллельно с веткой 3.x. Со вчерашнего дня и далее ветка Python 2.х обновляться больше",19,0
0,2021-07-06 10:29:57,Python Community Meetup 8/07: видео и материалы встречи,https://habr.com/ru/company/raiffeisenbank/news/t/566370/,"Первый открытый онлайн-митап сообщества, для которого собрали огонь-программу. Вместе со спикерами из Райффайзенбанка и Яндекса обсудили новости с полей — «Python ускоряется?», поговорили об использовании Dependency Injection и применении Python в highload",3,0
0,2022-01-13 15:35:30,"Открытый урок «Пишем Custom Prometheus Exporter на Python», 19 января",https://habr.com/ru/company/southbridge/news/t/645485/,"19 января Слёрм проведёт открытый урок «Пишем Custom Prometheus Exporter на Python». В главной роли Денис Наумов – спикер курса «Python для инженеров». Начало в 19:00 по Москве.Научим писать Prometheus Exporter для случаев, когда сервис не умеет работать п",11,1
0,2020-12-04 18:03:25,Python как компилируемый статически типизированный язык программирования,https://habr.com/ru/news/t/531402/,"По данным широко известного в узких кругах Tiobe Index язык Python скорее всего станет языком 2020 года, в четвертый раз в своей карьере. Кроме того, скорее всего он обгонит Java и займет вторую строчку в общем рейтинге языков программирования вслед за язы",22,6
0,2022-03-08 09:13:52,Вышел мартовский релиз расширения Python для Visual Studio Code,https://habr.com/ru/news/t/654707/,"Вышел выпуск расширения Python для Visual Studio Code за март 2022 года. Это короткий выпуск, но он включает некоторые новые функции, такие как поддержка предварительных версий и изменения в выборе интерпретатора в строке состояния.Полный список улучшений",0,0
0,2020-03-03 10:22:32,В начале этого года Python сместил Java и стал вторым по популярности языком программирования среди разработчиков,https://habr.com/ru/company/itsumma/news/t/490834/,"Согласно отчету RedMonk за январь 2020 года, Python стал вторым по популярности языком программирования после Java Script. Ранее эту позицию на протяжении длительного времени уверенно удерживал Java, однако в начале года этот ЯП сместился на третью строчку",31,0
0,2021-11-16 13:09:46,EPAM разработала бесплатный курс по программированию для детей и подростков «Chatbot on Python. Part 1»,https://habr.com/ru/company/epam_systems/news/t/589555/,Компания EPAM запустила бесплатный курс для самостоятельной подготовки «Chatbot on Python. Part 1: прокачай свой Python». Он доступен для всех желающих на платформе learn.epam.com в русской и английской версиях.Программа разработана для детей и подростков,3,0
0,2022-08-19 15:07:02,Осталась неделя до старта 4 потока Python для инженеров,https://habr.com/ru/company/southbridge/news/t/683604/,"29 августа в последний раз в этом году запустим 4 поток курса Python для инженеров. 3 месяца на практике будем постигать админские библиотеки и преисполнятся в